In [1]:
! pip install python-Levenshtein
import numpy as np
import scipy.stats as sps

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from functools import lru_cache
from Levenshtein import distance as levenshtein_distance
from collections import defaultdict
import numpy as np


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip '/content/drive/MyDrive/Colab_Notebooks/data/for_task.zip'

Archive:  /content/drive/MyDrive/Colab_Notebooks/data/for_task.zip
   creating: for_task/
  inflating: for_task/Barracuda.tsv  
  inflating: for_task/Boss.tsv       
  inflating: for_task/Dance_Till_Dead.tsv  
  inflating: for_task/FEV_Reject.tsv  
  inflating: for_task/Johny.tsv      
  inflating: for_task/King_Charles.tsv  
  inflating: for_task/Lucky_Number_5.tsv  
  inflating: for_task/Lychelle.tsv   
  inflating: for_task/Matchstick_Man.tsv  
  inflating: for_task/Mengsk.tsv     
  inflating: for_task/Pavlina_Grey.tsv  
  inflating: for_task/Starry_Sky.tsv  
  inflating: for_task/The_Wide_Pirate.tsv  
  inflating: for_task/vdjdb.slim.txt  
  inflating: for_task/Wing_And_A_Prayer.tsv  


# Прочтение таблиц

In [5]:
df = pd.read_csv("/content/for_task/Barracuda.tsv", sep='\t')
df.head()

,count,freq,cdr3nt,cdr3aa,v,d,j,VEnd,DStart,DEnd,JStart
0,256397,0.067121,TGCGCCAGCAGCCAAGATTGGGGGACAGACTCCCTATTCTCTGGAA...,CASSQDWGTDSLFSGNTIYF,"TRBV4-3,TRBV4-2,TRBV4-1",TRBD1,TRBJ1-3,18,21,28,38
1,137460,0.035985,TGTGCCAGCAGCCTCCAGACAGGGTTGAACACTGAAGCTTTCTTT,CASSLQTGLNTEAFF,"TRBV7-9,TRBV7-6",TRBD1,TRBJ1-1,12,17,24,25
2,66664,0.017452,TGTGCCAGCAGCTTAGTGGGGGGCGCGGGGGAGCAGTACTTC,CASSLVGGAGEQYF,"TRBV7-9,TRBV7-6",TRBD1,TRBJ2-7,16,18,26,30
3,63072,0.016511,TGTGCCAGCCCCTGAGCTAAATTAGAGAGCAGTACTTC,CASP*A_IREQYF,TRBV6-4,TRBD1,TRBJ2-7,9,9,14,26
4,57317,0.015005,TGTGCCAGCAGTTTATCGATTCGCAGGGCGGGCACTGAAGCTTTCTTT,CASSLSIRRAGTEAFF,TRBV28,"TRBD2,TRBD1",TRBJ1-1,16,23,28,32


In [8]:
database = pd.read_excel("/content/drive/MyDrive/Colab_Notebooks/data/vdjdb_slim.xlsx")

In [10]:
database_exp = database.rename(columns={'cdr3': 'CDR3.amino.acid.sequence'})

In [11]:
df = df.rename(columns={'cdr3aa': 'CDR3.amino.acid.sequence'})

In [13]:
df = df.sort_values(by='count', ascending=False)
df

,count,freq,cdr3nt,CDR3.amino.acid.sequence,v,d,j,VEnd,DStart,DEnd,JStart
0,256397,6.712129e-02,TGCGCCAGCAGCCAAGATTGGGGGACAGACTCCCTATTCTCTGGAA...,CASSQDWGTDSLFSGNTIYF,"TRBV4-3,TRBV4-2,TRBV4-1",TRBD1,TRBJ1-3,18,21,28,38
1,137460,3.598518e-02,TGTGCCAGCAGCCTCCAGACAGGGTTGAACACTGAAGCTTTCTTT,CASSLQTGLNTEAFF,"TRBV7-9,TRBV7-6",TRBD1,TRBJ1-1,12,17,24,25
2,66664,1.745174e-02,TGTGCCAGCAGCTTAGTGGGGGGCGCGGGGGAGCAGTACTTC,CASSLVGGAGEQYF,"TRBV7-9,TRBV7-6",TRBD1,TRBJ2-7,16,18,26,30
3,63072,1.651140e-02,TGTGCCAGCCCCTGAGCTAAATTAGAGAGCAGTACTTC,CASP*A_IREQYF,TRBV6-4,TRBD1,TRBJ2-7,9,9,14,26
4,57317,1.500482e-02,TGTGCCAGCAGTTTATCGATTCGCAGGGCGGGCACTGAAGCTTTCTTT,CASSLSIRRAGTEAFF,TRBV28,"TRBD2,TRBD1",TRBJ1-1,16,23,28,32
...,...,...,...,...,...,...,...,...,...,...,...
179182,2,5.235731e-07,TGCAGCGTTTGGGGATCTAGCGGGGGGTATTCCTACAATGAGCAGT...,CSVWGSSGGYSYNEQFF,TRBV29-1,TRBD2,TRBJ2-1,9,11,27,30
179183,2,5.235731e-07,TGCAGCGTTTTGACGGGGTGGGGGGACAGGGCCTCCTATGGCTACA...,CSVLTGWGDRASYGYTF,TRBV29-1,TRBD1,TRBJ1-2,9,22,31,35
179184,2,5.235731e-07,TGTGCCAGCAGCCCCCCGGGACAGGGGCGTACGCAGTTCTTC,CASSPPGQGRTQFF,TRBV13,TRBD1,TRBJ2-1,12,14,27,32
179185,2,5.235731e-07,TGTGCCAGCAGCCCCGGGTCAGGGGTCTACGAGCAGTACTTC,CASSPGSGVYEQYF,TRBV13,TRBD1,TRBJ2-7,12,12,25,26


In [15]:
split_df = df['v'].str.split(',', expand=True)

split_df.fillna(0, inplace=True)

split_df.columns = [f'bestVGene_{i+1}' for i in range(split_df.shape[1])]

df = pd.concat([df, split_df], axis=1)
df

,count,freq,cdr3nt,CDR3.amino.acid.sequence,v,d,j,VEnd,DStart,DEnd,JStart,bestVGene_1,bestVGene_2,bestVGene_3,bestVGene_4,bestVGene_5
0,256397,6.712129e-02,TGCGCCAGCAGCCAAGATTGGGGGACAGACTCCCTATTCTCTGGAA...,CASSQDWGTDSLFSGNTIYF,"TRBV4-3,TRBV4-2,TRBV4-1",TRBD1,TRBJ1-3,18,21,28,38,TRBV4-3,TRBV4-2,TRBV4-1,0,0
1,137460,3.598518e-02,TGTGCCAGCAGCCTCCAGACAGGGTTGAACACTGAAGCTTTCTTT,CASSLQTGLNTEAFF,"TRBV7-9,TRBV7-6",TRBD1,TRBJ1-1,12,17,24,25,TRBV7-9,TRBV7-6,0,0,0
2,66664,1.745174e-02,TGTGCCAGCAGCTTAGTGGGGGGCGCGGGGGAGCAGTACTTC,CASSLVGGAGEQYF,"TRBV7-9,TRBV7-6",TRBD1,TRBJ2-7,16,18,26,30,TRBV7-9,TRBV7-6,0,0,0
3,63072,1.651140e-02,TGTGCCAGCCCCTGAGCTAAATTAGAGAGCAGTACTTC,CASP*A_IREQYF,TRBV6-4,TRBD1,TRBJ2-7,9,9,14,26,TRBV6-4,0,0,0,0
4,57317,1.500482e-02,TGTGCCAGCAGTTTATCGATTCGCAGGGCGGGCACTGAAGCTTTCTTT,CASSLSIRRAGTEAFF,TRBV28,"TRBD2,TRBD1",TRBJ1-1,16,23,28,32,TRBV28,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179182,2,5.235731e-07,TGCAGCGTTTGGGGATCTAGCGGGGGGTATTCCTACAATGAGCAGT...,CSVWGSSGGYSYNEQFF,TRBV29-1,TRBD2,TRBJ2-1,9,11,27,30,TRBV29-1,0,0,0,0
179183,2,5.235731e-07,TGCAGCGTTTTGACGGGGTGGGGGGACAGGGCCTCCTATGGCTACA...,CSVLTGWGDRASYGYTF,TRBV29-1,TRBD1,TRBJ1-2,9,22,31,35,TRBV29-1,0,0,0,0
179184,2,5.235731e-07,TGTGCCAGCAGCCCCCCGGGACAGGGGCGTACGCAGTTCTTC,CASSPPGQGRTQFF,TRBV13,TRBD1,TRBJ2-1,12,14,27,32,TRBV13,0,0,0,0
179185,2,5.235731e-07,TGTGCCAGCAGCCCCGGGTCAGGGGTCTACGAGCAGTACTTC,CASSPGSGVYEQYF,TRBV13,TRBD1,TRBJ2-7,12,12,25,26,TRBV13,0,0,0,0


In [16]:
data_claster = df[:3000]
data_claster

,count,freq,cdr3nt,CDR3.amino.acid.sequence,v,d,j,VEnd,DStart,DEnd,JStart,bestVGene_1,bestVGene_2,bestVGene_3,bestVGene_4,bestVGene_5
0,256397,0.067121,TGCGCCAGCAGCCAAGATTGGGGGACAGACTCCCTATTCTCTGGAA...,CASSQDWGTDSLFSGNTIYF,"TRBV4-3,TRBV4-2,TRBV4-1",TRBD1,TRBJ1-3,18,21,28,38,TRBV4-3,TRBV4-2,TRBV4-1,0,0
1,137460,0.035985,TGTGCCAGCAGCCTCCAGACAGGGTTGAACACTGAAGCTTTCTTT,CASSLQTGLNTEAFF,"TRBV7-9,TRBV7-6",TRBD1,TRBJ1-1,12,17,24,25,TRBV7-9,TRBV7-6,0,0,0
2,66664,0.017452,TGTGCCAGCAGCTTAGTGGGGGGCGCGGGGGAGCAGTACTTC,CASSLVGGAGEQYF,"TRBV7-9,TRBV7-6",TRBD1,TRBJ2-7,16,18,26,30,TRBV7-9,TRBV7-6,0,0,0
3,63072,0.016511,TGTGCCAGCCCCTGAGCTAAATTAGAGAGCAGTACTTC,CASP*A_IREQYF,TRBV6-4,TRBD1,TRBJ2-7,9,9,14,26,TRBV6-4,0,0,0,0
4,57317,0.015005,TGTGCCAGCAGTTTATCGATTCGCAGGGCGGGCACTGAAGCTTTCTTT,CASSLSIRRAGTEAFF,TRBV28,"TRBD2,TRBD1",TRBJ1-1,16,23,28,32,TRBV28,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3034,57,0.000015,TGTGCCAGCCGATCGGGACAGCACCAAATCTATGGCTACACCTTC,CASRSGQHQIYGYTF,TRBV2,TRBD1,TRBJ1-2,9,13,21,29,TRBV2,0,0,0,0
3044,57,0.000015,TGCGCCAGCAGCCAAGGAAGGGGAGGGAGAGGCACCGGGGAGCTGT...,CASSQGRGGRGTGELFF,"TRBV4-3,TRBV4-2,TRBV4-1",TRBD2,TRBJ2-2,16,20,27,32,TRBV4-3,TRBV4-2,TRBV4-1,0,0
3043,57,0.000015,TGTGCCAGCCGGGGACAGTCGAACACCGGGGAGCTGTTTTTT,CASRGQSNTGELFF,TRBV9,TRBD1,TRBJ2-2,9,11,18,19,TRBV9,0,0,0,0
3042,57,0.000015,TGCAGTGCTAGAGACCCCCCCACTGAAGCTTTCTTT,CSARDPPTEAFF,TRBV20-1,TRBD1,TRBJ1-1,14,14,19,20,TRBV20-1,0,0,0,0


# Функция кластеризации

In [17]:
# Кэшированная функция для вычисления расстояния Левенштейна
@lru_cache(maxsize=None)
def cached_levenshtein_distance(seq1, seq2):
    return levenshtein_distance(seq1, seq2)

# Функция для кластеризации последовательностей
def cluster_sequences(sequences, max_distance=1):
    clusters = []
    for seq in sequences:
        added_to_cluster = False
        for cluster in clusters:
            if any(cached_levenshtein_distance(seq, existing_seq) <= max_distance for existing_seq in cluster):
                cluster.append(seq)
                added_to_cluster = True
                break
        if not added_to_cluster:
            clusters.append([seq])
    return clusters

# Функция для объединения кластеров
def merge_clusters(clusters, max_distance=1):
    merged_clusters = []
    for cluster in clusters:
        added_to_cluster = False
        for merged_cluster in merged_clusters:
            if any(cached_levenshtein_distance(seq, existing_seq) <= max_distance for seq in cluster for existing_seq in merged_cluster):
                merged_cluster.extend(cluster)
                added_to_cluster = True
                break
        if not added_to_cluster:
            merged_clusters.append(cluster)
    return merged_clusters

# Пример последовательностей аминокислот
sequences = data_claster['CDR3.amino.acid.sequence']

# Разделение данных на части
chunk_size = len(sequences) // 100
chunks = [sequences[i:i + chunk_size] for i in range(0, len(sequences), chunk_size)]

# Кластеризация каждой части
all_clusters = []
for chunk in chunks:
    all_clusters.extend(cluster_sequences(chunk, max_distance=1))

# Объединение кластеров
final_clusters = merge_clusters(all_clusters, max_distance=1)

# Вывод результатов
# Создание DataFrame
cluster_data = []
for i, cluster in enumerate(final_clusters):
    for seq in cluster:
        cluster_data.append({'Cluster': i + 1, 'Sequence': seq})

results_df = pd.DataFrame(cluster_data)

In [18]:
# Создание нового DataFrame с данными из data_claster и номером кластера
cluster_assignments = {}
for i, cluster in enumerate(final_clusters):
  for seq in cluster:
    cluster_assignments[seq] = i + 1

results_df = data_claster.copy()
results_df['Cluster'] = results_df['CDR3.amino.acid.sequence'].map(cluster_assignments)

# Вывод нового DataFrame results_df
results_df


,count,freq,cdr3nt,CDR3.amino.acid.sequence,v,d,j,VEnd,DStart,DEnd,JStart,bestVGene_1,bestVGene_2,bestVGene_3,bestVGene_4,bestVGene_5,Cluster
0,256397,0.067121,TGCGCCAGCAGCCAAGATTGGGGGACAGACTCCCTATTCTCTGGAA...,CASSQDWGTDSLFSGNTIYF,"TRBV4-3,TRBV4-2,TRBV4-1",TRBD1,TRBJ1-3,18,21,28,38,TRBV4-3,TRBV4-2,TRBV4-1,0,0,1
1,137460,0.035985,TGTGCCAGCAGCCTCCAGACAGGGTTGAACACTGAAGCTTTCTTT,CASSLQTGLNTEAFF,"TRBV7-9,TRBV7-6",TRBD1,TRBJ1-1,12,17,24,25,TRBV7-9,TRBV7-6,0,0,0,2
2,66664,0.017452,TGTGCCAGCAGCTTAGTGGGGGGCGCGGGGGAGCAGTACTTC,CASSLVGGAGEQYF,"TRBV7-9,TRBV7-6",TRBD1,TRBJ2-7,16,18,26,30,TRBV7-9,TRBV7-6,0,0,0,3
3,63072,0.016511,TGTGCCAGCCCCTGAGCTAAATTAGAGAGCAGTACTTC,CASP*A_IREQYF,TRBV6-4,TRBD1,TRBJ2-7,9,9,14,26,TRBV6-4,0,0,0,0,4
4,57317,0.015005,TGTGCCAGCAGTTTATCGATTCGCAGGGCGGGCACTGAAGCTTTCTTT,CASSLSIRRAGTEAFF,TRBV28,"TRBD2,TRBD1",TRBJ1-1,16,23,28,32,TRBV28,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3034,57,0.000015,TGTGCCAGCCGATCGGGACAGCACCAAATCTATGGCTACACCTTC,CASRSGQHQIYGYTF,TRBV2,TRBD1,TRBJ1-2,9,13,21,29,TRBV2,0,0,0,0,2826
3044,57,0.000015,TGCGCCAGCAGCCAAGGAAGGGGAGGGAGAGGCACCGGGGAGCTGT...,CASSQGRGGRGTGELFF,"TRBV4-3,TRBV4-2,TRBV4-1",TRBD2,TRBJ2-2,16,20,27,32,TRBV4-3,TRBV4-2,TRBV4-1,0,0,2827
3043,57,0.000015,TGTGCCAGCCGGGGACAGTCGAACACCGGGGAGCTGTTTTTT,CASRGQSNTGELFF,TRBV9,TRBD1,TRBJ2-2,9,11,18,19,TRBV9,0,0,0,0,2828
3042,57,0.000015,TGCAGTGCTAGAGACCCCCCCACTGAAGCTTTCTTT,CSARDPPTEAFF,TRBV20-1,TRBD1,TRBJ1-1,14,14,19,20,TRBV20-1,0,0,0,0,2829


# Подсчёт Total Read Proportion

In [19]:

# Подсчет количества последовательностей в каждом кластере
cluster_counts = {i + 1: len(cluster) for i, cluster in enumerate(final_clusters)}
# Подсчет суммарного Read.proportion для каждого кластера
cluster_proportions = {}
for index, row in data_claster.iterrows():
  seq = row['CDR3.amino.acid.sequence']
  proportion = row['freq']
  for i, cluster in enumerate(final_clusters):
    if seq in cluster:
      cluster_proportions[i + 1] = cluster_proportions.get(i + 1, 0) + proportion
      break

# Создание DataFrame с суммарными пропорциями, номерами кластеров и количеством последовательностей
proportions_data = []
for cluster_number, total_proportion in cluster_proportions.items():
  proportions_data.append({'Cluster': cluster_number,
               'Total Read Proportion': total_proportion,
               'Sequence Count': cluster_counts[cluster_number]})



# Сортировка по Sequence Count

In [21]:
proportions_df = pd.DataFrame(proportions_data)
proportions_df = proportions_df.sort_values(by='Sequence Count', ascending=False)
proportions_df

,Cluster,Total Read Proportion,Sequence Count
492,493,0.000435,20
65,66,0.000840,7
545,546,0.000150,7
204,205,0.000252,6
264,265,0.000270,6
...,...,...,...
984,985,0.000031,1
985,986,0.000031,1
986,987,0.000031,1
987,988,0.000031,1


# Сортировка по Total Read Proportion

In [22]:
proportions_df_sorted = proportions_df.sort_values(by='Total Read Proportion', ascending=False)
#proportions_df_sorted = proportions_df_sorted.loc[proportions_df_sorted['Sequence Count'] > 1]
proportions_df_sorted

,Cluster,Total Read Proportion,Sequence Count
0,1,0.067121,1
1,2,0.035985,1
2,3,0.017969,5
3,4,0.016511,1
4,5,0.015005,1
...,...,...,...
2824,2825,0.000015,1
2825,2826,0.000015,1
2826,2827,0.000015,1
2827,2828,0.000015,1


# Выбор кластера

In [23]:
full = pd.merge(results_df, database_exp , on='CDR3.amino.acid.sequence', how='inner')

In [24]:
full = full[['gene', 'Cluster', 'CDR3.amino.acid.sequence', 'freq', 'bestVGene_1',	'bestVGene_2', 'bestVGene_3', 'bestVGene_4', 'bestVGene_5', 'j', 'v.segm',	'j.segm', 'mhc.class', 'species', 'antigen.species', 'antigen.gene']]
full

,gene,Cluster,CDR3.amino.acid.sequence,freq,bestVGene_1,bestVGene_2,bestVGene_3,bestVGene_4,bestVGene_5,j,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
0,TRB,2,CASSLQTGLNTEAFF,0.035985,TRBV7-9,TRBV7-6,0,0,0,TRBJ1-1,TRBV7-9*01,TRBJ1-1*01,MHCI,HomoSapiens,CMV,pp65
1,TRB,3,CASSLVGGAGEQYF,0.017452,TRBV7-9,TRBV7-6,0,0,0,TRBJ2-7,TRBV7-9*01,TRBJ2-7*01,MHCI,HomoSapiens,SARS-CoV-2,ORF1ab
2,TRB,8,CASSLVGGNEQFF,0.007760,TRBV12-4,TRBV12-3,0,0,0,TRBJ2-1,TRBV7-2*01,TRBJ2-1*01,MHCI,HomoSapiens,EBV,BZLF1
3,TRB,8,CASSLVGGNEQFF,0.007760,TRBV12-4,TRBV12-3,0,0,0,TRBJ2-1,TRBV7-3*01,TRBJ2-1*01,MHCI,HomoSapiens,EBV,EBNA4
4,TRB,8,CASSLVGGNEQFF,0.007760,TRBV12-4,TRBV12-3,0,0,0,TRBJ2-1,"TRBV7-3*01,TRBV7-2*01",TRBJ2-1*01,MHCI,HomoSapiens,CMV,IE1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,TRB,2761,CASSGNEQFF,0.000015,TRBV12-4,TRBV12-3,0,0,0,TRBJ2-1,TRBV9*01,TRBJ2-1*01,MHCI,HomoSapiens,DENV,NS3
149,TRB,2784,CSARDRTGNGYTF,0.000015,TRBV20-1,0,0,0,0,TRBJ1-2,TRBV20-1*01,TRBJ1-2*01,MHCI,HomoSapiens,EBV,BMLF1
150,TRB,2784,CSARDRTGNGYTF,0.000015,TRBV20-1,0,0,0,0,TRBJ1-2,TRBV20-1*01,TRBJ1-2*01,MHCI,HomoSapiens,EBV,BZLF1
151,TRB,2784,CSARDRTGNGYTF,0.000015,TRBV20-1,0,0,0,0,TRBJ1-2,TRBV20-1*01,TRBJ1-2*01,MHCI,HomoSapiens,InfluenzaA,M


In [25]:
mask_v1 = full['bestVGene_1'].str[:7] == full['v.segm'].str[:7]
mask_v2 = full['bestVGene_2'].str[:7] == full['v.segm'].str[:7]
mask_v3 = full['bestVGene_3'].str[:7] == full['v.segm'].str[:7]
mask_v4 = full['bestVGene_4'].str[:7] == full['v.segm'].str[:7]

mask_v1_suffix = full['bestVGene_1'].str[:7] == full['v.segm'].str[-10:-3]
mask_v2_suffix = full['bestVGene_2'].str[:7] == full['v.segm'].str[-10:-3]
mask_v3_suffix = full['bestVGene_3'].str[:7] == full['v.segm'].str[-10:-3]
mask_v4_suffix = full['bestVGene_4'].str[:7] == full['v.segm'].str[-10:-3]

v_gene_mask = mask_v1 | mask_v2 | mask_v3 | mask_v4 | mask_v1_suffix | mask_v2_suffix | mask_v3_suffix | mask_v4_suffix

j_gene_mask = full['j'].str[:7] == full['j.segm'].str[:7]

final_mask = v_gene_mask & j_gene_mask

full_vj = full.loc[final_mask]

In [26]:
full_total_prop = pd.merge(proportions_df_sorted, full_vj , on='Cluster', how='inner')
full_total_prop

,Cluster,Total Read Proportion,Sequence Count,gene,CDR3.amino.acid.sequence,freq,bestVGene_1,bestVGene_2,bestVGene_3,bestVGene_4,bestVGene_5,j,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
0,2,0.035985,1,TRB,CASSLQTGLNTEAFF,0.035985,TRBV7-9,TRBV7-6,0,0,0,TRBJ1-1,TRBV7-9*01,TRBJ1-1*01,MHCI,HomoSapiens,CMV,pp65
1,3,0.017969,5,TRB,CASSLVGGAGEQYF,0.017452,TRBV7-9,TRBV7-6,0,0,0,TRBJ2-7,TRBV7-9*01,TRBJ2-7*01,MHCI,HomoSapiens,SARS-CoV-2,ORF1ab
2,3,0.017969,5,TRB,CASSLVGGAGEQYF,0.000433,TRBV7-9,TRBV7-6,0,0,0,TRBJ2-7,TRBV7-9*01,TRBJ2-7*01,MHCI,HomoSapiens,SARS-CoV-2,ORF1ab
3,15,0.003446,4,TRB,CASSLEGDQPQHF,0.000053,TRBV5-1,0,0,0,0,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,CMV,IE1
4,15,0.003446,4,TRB,CASSLEGDQPQHF,0.000053,TRBV5-1,0,0,0,0,TRBJ1-5,TRBV5-1*01,TRBJ1-5*01,MHCI,HomoSapiens,InfluenzaA,NP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,2034,0.000019,1,TRB,CASSSGYTEAFF,0.000019,TRBV6-5,TRBV6-8,TRBV6-1,0,0,TRBJ1-1,TRBV6-5*01,TRBJ1-1*01,MHCII,HomoSapiens,InfluenzaA,M1
70,2214,0.000018,1,TRB,CASSITGNTEAFF,0.000018,TRBV19,0,0,0,0,TRBJ1-1,TRBV19*01,TRBJ1-1*01,MHCI,HomoSapiens,CMV,IE1
71,2784,0.000015,1,TRB,CSARDRTGNGYTF,0.000015,TRBV20-1,0,0,0,0,TRBJ1-2,TRBV20-1*01,TRBJ1-2*01,MHCI,HomoSapiens,EBV,BMLF1
72,2784,0.000015,1,TRB,CSARDRTGNGYTF,0.000015,TRBV20-1,0,0,0,0,TRBJ1-2,TRBV20-1*01,TRBJ1-2*01,MHCI,HomoSapiens,EBV,BZLF1


In [27]:
value_counts = full_vj['Cluster'].value_counts()
value_counts

,count
Cluster,
162,8
66,7
93,7
176,6
2784,3
27,3
15,3
508,3
268,3


In [28]:
full_full = pd.merge(proportions_df_sorted,value_counts, on = 'Cluster', how='inner' )
full_full

,Cluster,Total Read Proportion,Sequence Count,count
0,2,0.035985,1,1
1,3,0.017969,5,2
2,15,0.003446,4,3
3,27,0.001799,2,3
4,66,0.000840,7,7
5,49,0.000810,2,2
6,493,0.000435,20,2
7,91,0.000370,1,1
8,93,0.000364,1,7
9,94,0.000353,1,1


In [51]:
0.035985/full_full['Total Read Proportion'].sum() # confidence доля от всех кластеров(по total read prop) * доля от всех антигенов (тут 1)

0.5546426939403001

# Обработка для одного выбранного кластера

In [52]:
results_df_one_cluster = results_df.loc[results_df['Cluster'] == 2]
results_df_one_cluster = results_df_one_cluster.rename(columns={'Sequence': 'CDR3.amino.acid.sequence'})
results_df_one_cluster

,count,freq,cdr3nt,CDR3.amino.acid.sequence,v,d,j,VEnd,DStart,DEnd,JStart,bestVGene_1,bestVGene_2,bestVGene_3,bestVGene_4,bestVGene_5,Cluster
1,137460,0.035985,TGTGCCAGCAGCCTCCAGACAGGGTTGAACACTGAAGCTTTCTTT,CASSLQTGLNTEAFF,"TRBV7-9,TRBV7-6",TRBD1,TRBJ1-1,12,17,24,25,TRBV7-9,TRBV7-6,0,0,0,2


In [53]:

full = pd.merge(results_df_one_cluster, database_exp , on='CDR3.amino.acid.sequence', how='inner')

In [54]:
full = full[['gene','Cluster', 'CDR3.amino.acid.sequence', 'freq', 'bestVGene_1',	'j', 'v.segm',	'j.segm', 'mhc.class', 'species', 'antigen.species', 'antigen.gene']]
full

,gene,Cluster,CDR3.amino.acid.sequence,freq,bestVGene_1,j,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
0,TRB,2,CASSLQTGLNTEAFF,0.035985,TRBV7-9,TRBJ1-1,TRBV7-9*01,TRBJ1-1*01,MHCI,HomoSapiens,CMV,pp65


# Разборки с V и J